# Practical Session 7: Planning in certain and uncertain environments

## Redes de Sensores y Sistemas Autónomos 
### Grado en Ingeniería de las Tecnologías de Telecomunicación
### Universidad de Sevilla

David Alejo Teissière

## Introduction

In this practical session we will develop two different path planners. First, we will obtain a continuous, collision-free path connecting two distant points: the current pose of our Turtlebot and the goal destination. In this first part, we will assume to have perfect knowledge of the environment between the starting point to the goal point.

Then, in the second part, we will make our Turtlebot to be able to explore an unknown area. To this end, we will need a SLAM algorithm (Simultaneous Localization and Mapping) and we will develop a Frontier-based exploration method that will automatically generate exploration targets.

To sum up, in this lesson we will learn to:

* Obtain a collision-free path in a known environment
* Configure a SLAM algorithm to generate in real-time a map of the environment.
* Autonomously explore an area with frontier-based exploration.

Let's go!

## Introduction to path planning

In this section, we will assume that we have perfect knowledge of the environment, by means of a map which is specified as an Occupancy Grid (see Fig. 1). The path planning algorithm will explore the map and will find a path that connects the starting and goal points. Therefore, the block diagram of our path planning stack, including path tracking, is depicted in Fig. 2. It also includes the exploration part that is explained at the second part of the practical session.

To launch the path planning part, we have developed the `planning.launch` file. See its contents. It basically starts the following three nodes (besides including the modules developed so far):


<figure style="text-align:center">
  <img src="images/inflated_maze.png" alt="" width=700>
  <figcaption>Fig. 1: Geometry map of the Turlebot3 Maze environment.  </figcaption>
</figure>

<figure style="text-align:center">
  <img src="images/navigation_diagram.png" alt="" width=700>
  <figcaption>Fig. 2: Block diagram of our system including path planning, path tracking, collision avoidance and exploration and mapping.  </figcaption>
</figure>

### Map server node

The map server node publishes the map information of the environment, which is stored as an image file. It needs the image and some metadata information in YAML format.

Each pixel of the image is a number between 0 and 100, which indicates:

* Obstacle-free pixel: if the value is less than a threshold (pixel < free_thresh)
* Obstacle pixel: each pixel that exceeds a different threshold (pixel > occupied_thres)
* Unknown: Each pixel exceeding 100 or which is not obstacle-free or obstacle.

For more information, please refer to the ROS wiki: [Link](http://wiki.ros.org/map_server)

In [ ]:
###### Example of map metadata ##########
image: testmap.png
resolution: 0.1
origin: [0.0, 0.0, 0.0]
occupied_thresh: 0.65
free_thresh: 0.196
negate: 0

### Navigation costmap with Costmap2D package

We have to process the map of the environment to be able to distinguish between safe points or collision points for our robot.

To this end, this node takes as its input the map of then environment, and then generates a costmap which is calculated by taking into account the geometry of the robot as specified in Fig. 3. 

<figure style="text-align:center">
  <img src="images/costmapspec.png" alt="" >
  <figcaption>Fig. 3: Inflation of obstacles according to the geometry of the robot. Note the different radius of the robot: inscribed and circumscribed.  </figcaption>
</figure>

If we apply the inflation of the map of Fig. 2 taking into account the geometry of the Turtlebot burger (radius of 15-20 cm) we obtain the costmap of Fig. 4.


<figure style="text-align:center">
  <img src="images/costmap.png" alt="" width=700>
  <figcaption>Fig. 4: Inflated maze costmap.  </figcaption>
</figure>





### Planning node: Djikstra Algorithm

Djikstra's algorithm is one type of forward search algorithm that propagates evenly in the space until the goal node is reached.

It considers the costs associated to get from one node to its neighbor nodes. It is able to get the optimal paths assuming the costs to be nonnegative.

Our planning node gets the costmap of the environment and then generates a collision-free path connecting the starting and goal nodes, as shown in Fig. 5.

<figure style="text-align:center">
  <img src="images/djikstra.png" alt="" width=700>
  <figcaption>Fig. 5: Turtlebot maze scenario. The path obtained with Djikstra algorithm is in a green line.  </figcaption>
</figure>


In [ ]:
__Exercise_1__

Try to run the navigation experiment following the instructions of the professor. To this end you should:

1. Create the `navigation` catkin package that will be used in this session. It will depend on rospy, nav_msgs, sensor_msgs, geometry_msgs
2. In the practica_6.launch file, remap the PoseStamped subscriber of your Path Tracker to another topic (instead of move_base_simple/goal) 
3. Remove the visualizer node from practica_6.launch
4. Launch the planner.launch file of the navigation package 
5. Select a destination in the RViz file
6. See if the path is generated 
7. Check if the Turtlebot follows the generated path


__Exercise_2__

Convert the Djikstra algorithm into the A* algorithm by adding a heuristic cost to the goal node. Usually, the most commonly used heuristic is the euclidean distance from the current node to the goal, which can be obtained by using their coordinates.

Compare the running time of both algorithms (Djisktra and A*), which one is faster?

In [ ]:
__Exercise_3__

There are some improvements that can be made so that this node can be used in autonomous exploration nodes:

* We currently only check the obstacles at the beginning and consider as obstacles those cells exceeding a threshold. We should use the costmap itself in an online basis and check for obstacles in that map itself.
* Implement an action server to get a PoseStamped goal. 

### Autonomous Exploration

In this part of the practical session, we will assume that we deploy our Turtlebot in an environment which is unknown to her. 

#### Simultaneous localization and mapping (SLAM)

Thus, in this case we cannot assume that we have an a priori map of the environment. Instead, we will run a Simultaneous Localization and Mapping algorithm to let her acquire knowledge of the environment thanks to the information gathered by her onboard 2D LiDAR sensor. To this end, we can use the `gmapping` package available on ROS. This package generates some hypothesis each one consisting of not only the approximate location of the Turtlebot in the generated map, but also a map itself. The map is stored in a grid similar to the one used in the map server package."

More information: The gmapping package is based on MonteCarlo SLAM algorithm with Rao-Blackwellized particles. Please refer to the [paper](http://www2.informatik.uni-freiburg.de/~stachnis/pdf/grisetti05icra.pdf).

#### Frontier based exploration

The most simple yet powerful approach for autonomously exploring an area was proposed by B. Yamaouchi in his famous paper "[A Frontier-Based Approach for Autonomous Navigation](https://www.cs.cmu.edu/~motionplanning/papers/sbp_papers/integrated1/yamauchi_frontiers.pdf)" (1997). Quoting from him:

* "The central question in exploration is: Given what you know about the world, where should you move to gain as much new information as possible?"
* "The central idea behind frontier-based exploration is: To gain the most new information about the world, move to the boundary between open space and uncharted territory (i.e. frontiers)".
* "Frontiers are region on the boundary between open space and unexplored space"
* " Once frontiers have been detected within a particular evidence grid, the robot attempts to navigate to the nearest accessible, unvisited frontier."

Another interesting paper: "[Frontier based exploration for Autonomous Robots](https://arxiv.org/pdf/1806.03581)" by A. Topiwala et al.


#### Launching it all

The basic algorithms for SLAM and exploration are implemented in basic nodes of ROS. We use the following packages:

* gmapping: Implements the Rao-Blackwellized Particle Filter for SLAM
* move_base: Implements an action server for planning using Dynamic Window Approach (DWA)
* explore_lite: You have to download the source code from [here](https://github.com/hrnr/m-explore) into your catkin workspace and compile it (catkin_make)


Once you have download and compiled the exploration code, please download the new launch file (autonomous_exploration.launch). You can launch the autonomous exploration suite by using:

`(rssa_docker) > roslaunch navigation autonomous_exploration.launch `

#### Final remarks

At the end of the experiment, we can save the map progress by using the `map_saver` tool of the `map_server` package.

`(rssa_docker)> rosrun map_server map_saver`